In [1]:
import xgboost as xgb 
import pandas as pd 
import numpy as np
import os
from tqdm import tqdm

In [2]:
bid_model = xgb.XGBClassifier()
bid_model.load_model("../model_weight/bid_using.model")
ask_model = xgb.XGBClassifier()
ask_model.load_model("../model_weight/ask_using.model")

In [3]:
PATH = r"../stock_dataset/2330"
DATE = "20221202"

FILE_PATH = os.path.join(PATH,DATE+".csv")
data = pd.read_csv(FILE_PATH)


HOLD_CAPITAL = 10000
HOLD_STOCK = 0

# buy = 0 , sell = 1 ,nothing = -1
predict_action = -1 

loop = tqdm(range(len(data)))
for idx in loop :
    
    tick = data.iloc[idx].to_numpy()
    
    cur_match_price = tick[2]

    # buy
    if predict_action == 0 :

        HOLD_STOCK = HOLD_CAPITAL/cur_match_price
        HOLD_CAPITAL = 0
    # sell 
    elif predict_action == 1:
        HOLD_CAPITAL = HOLD_STOCK*cur_match_price
        HOLD_STOCK = 0 


    input_data = [[tick[3]]]    
    input_data[0].extend(tick[11:16])
    input_data[0].extend(tick[21:26])

    if HOLD_CAPITAL == 0:
        output = bid_model.predict_proba(input_data)
        choose = np.argmax(output[0])
        
        if choose == 1 :
            predict_action= 1
        else :
            predict_action = -1

    elif HOLD_STOCK == 0:
        output = ask_model.predict_proba(input_data)
        choose = np.argmax(output[0])

        if choose == 1 :
            predict_action= 0
        else :
            predict_action = -1
    loop.set_description(f"[{idx}/{len(data)}] {DATE} , capital: {HOLD_CAPITAL} , stock: {HOLD_STOCK}")
    loop.update
    if idx == len(data)-1:
        final_profit =HOLD_CAPITAL+HOLD_STOCK*cur_match_price
        print(f"final profit: {final_profit} , ROI: {final_profit/10000:2%}")
    




[7183/7184] 20221202 , capital: 0 , stock: 56.63104251167615: 100%|██████████| 7184/7184 [04:23<00:00, 27.29it/s] 

final profit: 27919.103958256343 , ROI: 279.191040%
